In [2]:
# 1. Import necessary libraries
import pandas as pd
import numpy as np

# 2. Load data
df = pd.read_csv("../data/raw/combine_data_2014_2024.csv")
print(f"Loaded shape: {df.shape}")
df.head()

Loaded shape: (3430, 18)


,season,draft_year,draft_team,draft_round,draft_ovr,pfr_id,cfb_id,player_name,pos,school,ht,wt,forty,bench,vertical,broad_jump,cone,shuttle
0,2014,2014.0,Green Bay Packers,5.0,176.0,AbbrJa00,jared-abbrederis-1,Jared Abbrederis,WR,Wisconsin,6-1,195.0,4.50,4.0,30.5,117.0,6.80,4.08
1,2014,2014.0,Green Bay Packers,2.0,53.0,AdamDa01,davante-adams-1,Davante Adams,WR,Fresno State,6-1,212.0,4.56,14.0,39.5,123.0,6.82,4.30
2,2014,2014.0,St. Louis Rams,4.0,110.0,AlexMa00,maurice-alexander-1,Mo Alexander,S,Utah State,6-1,220.0,4.50,NaN,38.0,123.0,7.05,4.51
3,2014,2014.0,Atlanta Falcons,5.0,147.0,AlleRi00,ricardo-allen-1,Ricardo Allen,CB,Purdue,5-9,187.0,4.61,13.0,35.5,117.0,NaN,4.15
4,2014,2014.0,New York Jets,2.0,49.0,AmarJa00,jace-amaro-1,Jace Amaro,TE,Texas Tech,6-5,265.0,4.74,28.0,33.0,118.0,7.42,4.30


In [3]:
# 3. Drop rows without 'draft_year'
# Since we are interested in players who were drafted, we need to remove players that don't have a draft year.
df.dropna(subset=['draft_year'], inplace=True)

# Verify if any rows were dropped
print(f"Shape after dropping rows without draft year: {df.shape}")

Shape after dropping rows without draft year: (2136, 18)


In [4]:
# 4. Impute numeric combine metrics
# For the columns with missing numeric values, we'll use the median to impute the missing values.
numeric_columns = ['forty', 'bench', 'vertical', 'broad_jump', 'cone', 'shuttle']
for col in numeric_columns:
    df[col].fillna(df[col].median(), inplace=True)

# Verify the imputation
print("Missing values after imputation:")
print(df[numeric_columns].isnull().sum())

Missing values after imputation:
forty         0
bench         0
vertical      0
broad_jump    0
cone          0
shuttle       0
dtype: int64


In [5]:
# 5. Parse 'ht' column into inches
# Convert height from feet-inches format (e.g., '6-1') to inches (e.g., 73)
def parse_height(height_str):
    if isinstance(height_str, str):
        feet, inches = height_str.split('-')
        return int(feet) * 12 + int(inches)
    return np.nan

df['ht_inches'] = df['ht'].apply(parse_height)

# Drop original 'ht' column
df.drop(columns=['ht'], inplace=True)

# Verify if the parsing was successful
print(f"Head after parsing height:\n{df[['player_name', 'ht_inches']].head()}")

Head after parsing height:
        player_name  ht_inches
0  Jared Abbrederis       73.0
1     Davante Adams       73.0
2      Mo Alexander       73.0
3     Ricardo Allen       69.0
4        Jace Amaro       77.0


In [6]:
# 6. Rename numeric columns to include units
df.rename(columns={
    'ht_inches': 'ht_inch',
    'wt': 'wt_lbs',
    'forty': 'forty_time_s',
    'bench': 'bench_reps',
    'vertical': 'vertical_in',
    'broad_jump': 'broad_jump_in',
    'cone': 'cone_sec',
    'shuttle': 'shuttle_sec'
}, inplace=True)

# Verify renaming
print(f"Columns after renaming:\n{df.columns}")


Columns after renaming:
Index(['season', 'draft_year', 'draft_team', 'draft_round', 'draft_ovr',
       'pfr_id', 'cfb_id', 'player_name', 'pos', 'school', 'wt_lbs',
       'forty_time_s', 'bench_reps', 'vertical_in', 'broad_jump_in',
       'cone_sec', 'shuttle_sec', 'ht_inch'],
      dtype='object')


In [ ]:
# Handle outliers using IQR method
def cap_outliers(df, cols):
    for col in cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df[col] = df[col].clip(lower, upper)
    return df

combine_metrics = ['wt', 'forty_time_s', 'bench', 'vertical', 'broad_jump', 'cone', 'shuttle']
df = cap_outliers(df, combine_metrics)


In [ ]:
#BMI = weight (lb) / height (in)^2 × 703
df['bmi'] = df['wt'] / (df['ht_inches'] ** 2)

In [ ]:
#Speed Score = (Weight * 200) / (40 time ^ 4)
df['speed_score'] = (df['wt'] * 200) / (df['forty_time_s'] ** 4)

In [ ]:
#Burst = Vertical Jump (in) + Broad Jump (in)
df['burst_score'] = df['vertical'] + df['broad_jump']

In [ ]:
from sklearn.preprocessing import StandardScaler

scale_cols = ['wt', 'forty_time_s', 'bench', 'vertical', 'broad_jump', 
              'cone', 'shuttle', 'bmi', 'speed_score', 'burst_score']

scaler = StandardScaler()
df[scale_cols] = scaler.fit_transform(df[scale_cols])


In [ ]:
#Drafted/Undrafted Indicator
df['was_drafted'] = df['draft_round'].notna().astype(int)

In [7]:
# 7. Save the cleaned data to a new CSV for later use
processed_data_path = "../data/processed/combine_preprocessed.csv"
df.to_csv(processed_data_path, index=False)

# Confirm data saved
print(f"Preprocessed data saved to: {processed_data_path}")

# 8. Final check of the data
print(f"Final shape of preprocessed data: {df.shape}")
df.head()

Preprocessed data saved to: ../data/processed/combine_preprocessed.csv
Final shape of preprocessed data: (2136, 18)


,season,draft_year,draft_team,draft_round,draft_ovr,pfr_id,cfb_id,player_name,pos,school,wt_lbs,forty_time_s,bench_reps,vertical_in,broad_jump_in,cone_sec,shuttle_sec,ht_inch
0,2014,2014.0,Green Bay Packers,5.0,176.0,AbbrJa00,jared-abbrederis-1,Jared Abbrederis,WR,Wisconsin,195.0,4.50,4.0,30.5,117.0,6.80,4.08,73.0
1,2014,2014.0,Green Bay Packers,2.0,53.0,AdamDa01,davante-adams-1,Davante Adams,WR,Fresno State,212.0,4.56,14.0,39.5,123.0,6.82,4.30,73.0
2,2014,2014.0,St. Louis Rams,4.0,110.0,AlexMa00,maurice-alexander-1,Mo Alexander,S,Utah State,220.0,4.50,20.0,38.0,123.0,7.05,4.51,73.0
3,2014,2014.0,Atlanta Falcons,5.0,147.0,AlleRi00,ricardo-allen-1,Ricardo Allen,CB,Purdue,187.0,4.61,13.0,35.5,117.0,7.16,4.15,69.0
4,2014,2014.0,New York Jets,2.0,49.0,AmarJa00,jace-amaro-1,Jace Amaro,TE,Texas Tech,265.0,4.74,28.0,33.0,118.0,7.42,4.30,77.0
